In [327]:
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import functools
import itertools
import operator
import os
import os.path as op
import numpy as np
import pytrends
from pytrends.request import TrendReq as UTrendReq
from datetime import date
import datetime as d
from collections import Counter
import click
import pdb
import sys, traceback

In [328]:
from dash import Dash, html, dcc, Input, Output
import pandas as pd
import plotly.express as px


In [329]:
# Opening JSON file
f = open('../data/raw/CMS_2010_to_June_2022_ENGLISH.json')
  
# returns JSON object as a dictionary
data = json.load(f)

# convert to data frame
df = pd.DataFrame.from_dict(data)

df['keywordStringsCleanAfterFuzz'] = df['keywordStrings']


In [330]:
df_clean = df[['id', 'lastModifiedDate', 'keywordStringsCleanAfterFuzz']]
df_clean.index = df_clean.index.astype('int64')

In [331]:
df_clean.index

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            175649, 175650, 175651, 175652, 175653, 175654, 175655, 175656,
            175657, 175658],
           dtype='int64', length=175659)

In [332]:
len(data)

28

In [333]:
#, get_dw_timeseries, cli_dw_vs_google

In [335]:
#keyword_set = {x for l in df['keywordStrings'] for x in l}
#from pytrends.request import TrendReq

In [336]:
# def get_interest_over_time_FT(keyword, start_date = '2019-01-01', end_date=f'{date.today()}'):
#     #keywords needs to be a list 
#     #need to make sure the total number of characters is less than 100 for Google and terms are fewer than 5 
#     # terms = 0
#     # chars = 0
#     # for word in keywords:
#     #     chars += len(word)
#     #     if chars > 99:
#     #         break
#     #     else:
#     #         terms += 1
#     # if terms > 5:
#     #     terms = 5
#     # keywords = keywords[:terms]
#     print(keyword)
#     if len(keyword)>99:
#         print('KEYWORD IS TOO LONG FOR THIS SEARCH')
#         return None
    
#     #let's get python trends 
#     pytrend = TrendReq(                    
#     # proxies=['https://34.203.233.13:80','https://35.201.123.31:880'], 
#     # #hl='en-US', tz=360, timeout=(10,25)
#     retries=2, backoff_factor=0.1, requests_args={'verify':False})
#     google_df = pytrend.build_payload(kw_list= [keyword], timeframe= '{} {}'.format(str(start_date),str(end_date)))
#     google_df = pytrend.interest_over_time()
#     if 'isPartial' in google_df.columns:
#         google_df = google_df.drop('isPartial', axis = 'columns')
        
#     fig = plt.plot(google_df, color = 'r', alpha =0.5)
#     plt.show()
#     return fig

In [356]:
# def get_interest_over_time_plot(google_df,dw_mentions,selected_keyword):
#     fig = px.line(x=google_df.resample('1W').mean()[selected_keyword].index ,y=[google_df.resample('1W').mean()[selected_keyword],dw_mentions]).update_layout(xaxis_title="Date")
#     plt.show()
#     return fig

In [364]:
import plotly.graph_objects as go

def get_interest_over_time_plot(google_df,dw_mentions,selected_keyword):
    fig = go.Figure()

    fig.add_trace(go.Scatter(
    x=google_df.resample('1W').mean()[selected_keyword].index,
    y=google_df.resample('1W').mean()[selected_keyword],
    name="Google Search Weekly Relative Average"       # this sets its legend entry
    ))


    fig.add_trace(go.Scatter(
    x=google_df.resample('1W').mean()[selected_keyword].index,
    y=dw_mentions,
    name="Number of DW articles per week"
    ))

    fig.update_layout(
    title="Plot Title",
    xaxis_title="Date",
    yaxis_title="Y Axis Title",
    legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"))

    fig.show()
    return fig



In [365]:
# google_searches = get_interest_over_time(keyword, start_date = start_date, end_date=end_date)

#     if google_searches.empty: 
#         print('Exiting...')
#         return -1
#     #datetime_object saved in column 'dt_lastModifiedDate' of df
#     # pdb.set_trace()
#     dw_mentions = dw_mentions[:google_searches.values.shape[0]] #REMOVE THIS LATER
#     assert dw_mentions.shape[0] == google_searches.values.shape[0]
#     #TODO: we need a perfect way for matching dates between the two
#     # at the moment we have year+no_of_week in dw and actual date on google 01.01.19, 08.01.2019  etc.)

#     #now let's compare for this keyword:
#     # 1) let's make plot 
#     fig, ax = plt.subplots(figsize=(15,10))
#     fig.autofmt_xdate(rotation=75)

#     # pdb.set_trace()
#     # plot_signals(dw_mentions, google_searches, ax, keyword = keyword)
#     #TODO: removee this and do it using function 
#     ax.plot(dw_mentions.index.astype(str), dw_mentions.val, color = 'k')
#     ax.set_xticks(dw_mentions.index.astype(str)[::8])
#     ax.set_xlabel('Time', fontsize = 20)
#     ax.set_ylabel(f'DW Articles per week with {keyword} in keywords', color = 'k', fontsize = 20)
#     ax2 = ax.twinx()
#     ax2.plot(dw_mentions.index.astype(str), google_searches.values, color = 'r', alpha =0.5)
#     ax2.set_ylabel(f'Relative amount of Google Searches for {keyword}', color = 'r', fontsize = 20)

In [366]:
def get_dw_timeseries_FT(df_clean, keyword, start = '2019-01-01', end='2019-06-01'):

    not_keyword_indices = [] #TODO: do this without a loop (create extra boolean column and assign True if keyword is there)
    for i, row in enumerate(df_clean['keywordStringsCleanAfterFuzz']):
        if keyword not in row:
            not_keyword_indices.append(i)

    df_clean = df_clean.drop(not_keyword_indices)
    df_clean['lastModifiedDate'] = pd.to_datetime(df_clean['lastModifiedDate'])
    df_clean.set_index('lastModifiedDate', inplace = True)
    df_clean = df_clean.loc[start:end]
    output = df_clean.resample('1W').count()['id']
    return output

In [367]:
# keyword='Germany'
# not_keyword_indices = [] #TODO: do this without a loop (create extra boolean column and assign True if keyword is there)
# for i, row in enumerate(df_clean['keywordStringsCleanAfterFuzz']):
#     if keyword not in row:
#         not_keyword_indices.append(i)

# df_clean = df_clean.drop(not_keyword_indices)

In [368]:
df_clean.index

Int64Index([     0,      1,      2,      3,      4,      5,      6,      7,
                 8,      9,
            ...
            175649, 175650, 175651, 175652, 175653, 175654, 175655, 175656,
            175657, 175658],
           dtype='int64', length=175659)

In [369]:
sys.path.append('..')

from models.MA_dw_to_google_pipeline import get_interest_over_time, get_dw_timeseries

app = Dash()

keyword_dropdown = dcc.Dropdown(options=['Germany', 'Donald Trump'], value='Germany')

app.layout = html.Div(children=[
    html.H1(children='Google Search Interest'),
    keyword_dropdown,
    dcc.Graph(id='search-graph')
])
@app.callback(
    Output(component_id='search-graph', component_property='figure'),
    Input(component_id=keyword_dropdown, component_property='value')
)
def update_graph(selected_keyword):
    google_df = get_interest_over_time(selected_keyword, start_date = '2019-01-01', end_date='2019-06-01')
    dw_mentions = get_dw_timeseries_FT(df_clean, selected_keyword, start = '2019-01-01', end='2019-06-01')
    line_fig = get_interest_over_time_plot(google_df,dw_mentions,selected_keyword)
    return line_fig

# Run local server
app.run_server(debug=True, port=8049, use_reloader=False)

Dash is running on http://127.0.0.1:8049/

 * Serving Flask app '__main__'
 * Debug mode: on
Germany


/home/ferdinand_t/venv/lib64/python3.7/site-packages/urllib3/connectionpool.py:1052: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings

/home/ferdinand_t/venv/lib64/python3.7/site-packages/urllib3/connectionpool.py:1052: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings

/home/ferdinand_t/venv/lib64/python3.7/site-packages/urllib3/connectionpool.py:1052: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings

/home/ferdinand_t/venv/lib64/python3.7/site-packages/urllib3/connec

Germany


/home/ferdinand_t/venv/lib64/python3.7/site-packages/urllib3/connectionpool.py:1052: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings

/home/ferdinand_t/venv/lib64/python3.7/site-packages/urllib3/connectionpool.py:1052: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings

/home/ferdinand_t/venv/lib64/python3.7/site-packages/urllib3/connectionpool.py:1052: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'trends.google.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings

/home/ferdinand_t/venv/lib64/python3.7/site-packages/urllib3/connec